In [0]:
import pandas as pd
import numpy as np
import keras
# from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate, StratifiedKFold, train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
# from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import pickle
import time
import tensorflow as tf

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
def f1(y_true, y_pred):
    y_pred = np.rint(y_pred)
    tp = sum(y_true*y_pred)
    tn = sum((1-y_true)*(1-y_pred))
    fp = sum((1-y_true)*y_pred)
    fn = sum(y_true*(1-y_pred))

    p = tp / (tp + fp )
    r = tp / (tp + fn )

    f1 = 2*p*r / (p+r)
    return f1

In [0]:
#def f1_loss(y_true, y_pred):
#    
#    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
#    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
#    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
#    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)
#
#    p = tp / (tp + fp + K.epsilon())
#    r = tp / (tp + fn + K.epsilon())
#
#    f1 = 2*p*r / (p+r+K.epsilon())
#    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
#    return  1 - K.mean(f1)

In [0]:
start_time = time.time()
y = pd.read_csv("/content/gdrive/My Drive/data/y_bush_vs_others.csv", header= None)
y_bush = y.values.ravel()
y = pd.read_csv("/content/gdrive/My Drive/data/y_williams_vs_others.csv", header= None)
y_williams = y.values.ravel()
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.019988059997558594 seconds ---


In [0]:
start_time = time.time()
X = pd.read_csv("/content/gdrive/My Drive/data/X.csv", sep=' ', header= None, dtype= float)
X = X.values
print("--- %s seconds ---" % (time.time() - start_time))

--- 24.694132089614868 seconds ---


In [0]:
X_train, X_test, y_bush_train, y_bush_test = train_test_split(X,y_bush, test_size=1./3, random_state = 6926, shuffle = True, stratify = y_bush)

In [0]:
X_train.shape

(8822, 4096)

In [0]:
X_test.shape

(4411, 4096)

In [0]:
X.shape

(13233, 4096)

In [0]:
X_train = X_train.reshape(8822,64,64,1)

In [0]:
X_test = X_test.reshape(4411,64,64,1)

In [0]:
y_bush_train = y_bush_train.reshape(8822,1)
y_bush_test = y_bush_test.reshape(4411,1)


In [0]:
y_bush_train.shape

(8822, 1)

In [0]:
X_train.shape

(8822, 64, 64, 1)

In [0]:
X_test.shape

(4411, 64, 64, 1)

In [0]:
bush_model = Sequential()

In [0]:
bush_model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation='relu',input_shape=(64,64,1)))
bush_model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation='relu'))
bush_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

bush_model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
bush_model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
bush_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

bush_model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
bush_model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
bush_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

bush_model.add(Flatten())
bush_model.add(Dense(1000,activation='relu'))
bush_model.add(Dense(1,activation='sigmoid'))

In [0]:
bush_model.compile(loss=keras.losses.binary_crossentropy, optimizer='adam', metrics=['accuracy'])

In [0]:
csv_logger = keras.callbacks.CSVLogger('/content/gdrive/My Drive/data/bush_training.log')
bush_model.fit(X_train, y_bush_train, batch_size = 500, epochs = 10, verbose =1 ,callbacks=[csv_logger])
print(f1(y_bush_train,bush_model.predict(X_train)))
print(f1(y_bush_test,bush_model.predict(X_test)))

Epoch 1/10
8822/8822 [==============================] - 3s 389us/step - loss: 0.0232 - acc: 0.9930
Epoch 2/10
8822/8822 [==============================] - 3s 379us/step - loss: 0.0159 - acc: 0.9951
Epoch 3/10
8822/8822 [==============================] - 3s 380us/step - loss: 0.0141 - acc: 0.9952
Epoch 4/10
8822/8822 [==============================] - 3s 378us/step - loss: 0.0163 - acc: 0.9937
Epoch 5/10
8822/8822 [==============================] - 3s 378us/step - loss: 0.0132 - acc: 0.9954
Epoch 6/10
8822/8822 [==============================] - 3s 378us/step - loss: 0.0082 - acc: 0.9976
Epoch 7/10
8822/8822 [==============================] - 3s 378us/step - loss: 0.0050 - acc: 0.9988
Epoch 8/10
8822/8822 [==============================] - 3s 377us/step - loss: 0.0028 - acc: 0.9993
Epoch 9/10
8822/8822 [==============================] - 3s 378us/step - loss: 0.0015 - acc: 0.9998
Epoch 10/10
8822/8822 [==============================] - 3s 377us/step - loss: 0.0010 - acc: 0.9998
[1.]
[0.7

In [0]:
bush_model.fit(X_train, y_bush_train, batch_size = 500, epochs = 10, verbose =1 ,callbacks=[csv_logger])
print(f1(y_bush_train,bush_model.predict(X_train)))
print(f1(y_bush_test,bush_model.predict(X_test)))

In [0]:
bush_model.save("/content/gdrive/My Drive/data/bush.model")

In [0]:
pickle.dump([f1(y_bush_train,bush_model.predict(X_train))[0],f1(y_bush_test,bush_model.predict(X_test))[0]], open('/content/gdrive/My Drive/data/bush.pkl', 'wb'))

In [0]:
pickle.load(open('/content/gdrive/My Drive/data/bush.pkl', 'rb'))

[1.0, 0.7784431137724552]